In [30]:
import requests
import bs4
from bs4 import BeautifulSoup as bs
import csv
import re
import numpy as np
import pandas as pd
import nltk


Get the content for the root forum page.

In [39]:
canadavisa_url = 'https://www.canadavisa.com/canada-immigration-discussion-board/'
canadavisa = requests.get(canadavisa_url).text

Get the 

In [43]:
soup = bs(canadavisa, 'html.parser')
topics = soup.find_all('h3', {'class':'nodeTitle'})
topic_titles = list(map(lambda h: h.text.strip(), topics))
topic_titles_and_links = dict(map(lambda h: (h.text.strip(), h.find('a') ['href']), topics))

# print(titles_and_links)

Creating a csv file to open the data in an excel sheet.

In [36]:
with open('titles_and_links.csv', mode='w') as csv_file:
    fieldnames = ['title', 'link']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()
    for key, value in titles_and_links.items():
        writer.writerow({'title': key, 'link': value})


Iterate through the topic titles to extract threads and messages.

In [45]:
# Create lists to hold the messages.
message_contents = list()
date_contents = list()
username_contents = list()
topic_contents = list()
subtopic_contents = list()

counter = 0


for key, value in topic_titles_and_links.items():
    counter = counter + 1
    if counter == 1:
        print('inside loop', key, value)
        continue

    # For debugging only - COMMENT OUT LATER
    if counter > 2:
        print('inside loop',key,value)
        continue

    topic_url = Canadavisa_url + value 
    topic_title = key
    try:
        topic_content = bs(requests.get(topic_url).text, 'html.parser')
        threads = topic_content.find_all('h3', {'class':'title'})
        thread_titles_and_links = dict(map(lambda subtopic: (subtopic.text.strip(), subtopic.find('a') ['href']), threads))
        thread_counter = 0
        for st_key, st_value in thread_titles_and_links.items():
            thread_counter = thread_counter + 1
            if thread_counter > 1:
                print('inside loop',key,value)
                continue
            thread_url = Canadavisa_url + st_value
            thread_title = st_key
            thread_content = bs(requests.get(subtopic_url).text, 'html.parser')
            thread_pages = thread_content.find_all('nav')
            for thread_page in thread_pages:
                link = thread_page.find('a', {'class': 'PageNavNext'})
                print(link)
    except:
        print("Can't find this page... Moving on")



inside loop Immigration to Canada .#immigration-to-canada.9
None
None
None
<a class="PageNavNext ">→</a>
<a class="PageNavNext ">→</a>
None
inside loop Express Entry / Expression of Interest forums/express-entry-expression-of-interest.54/
inside loop Express Entry / Expression of Interest forums/express-entry-expression-of-interest.54/
inside loop Express Entry / Expression of Interest forums/express-entry-expression-of-interest.54/
inside loop Express Entry / Expression of Interest forums/express-entry-expression-of-interest.54/
inside loop Express Entry / Expression of Interest forums/express-entry-expression-of-interest.54/
inside loop Express Entry / Expression of Interest forums/express-entry-expression-of-interest.54/
inside loop Express Entry / Expression of Interest forums/express-entry-expression-of-interest.54/
inside loop Express Entry / Expression of Interest forums/express-entry-expression-of-interest.54/
inside loop Express Entry / Expression of Interest forums/express-en

In [127]:
int(re.search('Page \d* of (\d*)', 'Page 1 of 1280').group(1))

1280

In [130]:
message_ids = list()
links = list()
date_contents = list()
username_contents = list()
message_contents = list()
countries = list()
likes = list()
topic_contents = list()
thread_contents = list()
counter = 0
thread_counter = 0
thread_page_counter = 0

for key, value in topic_titles_and_links.items():
    counter = counter + 1
    if counter == 1:
        print('inside loop',key,value)
        continue

    # For debugging only - COMMENT OUT LATER
    if counter > 2:
        break

    
    topic_url = Canadavisa_url + value 
    topic_title = key

    print("Processing ", topic_title)
    try:
        topic_content = bs(requests.get(topic_url).text, 'html.parser')
        threads = topic_content.find_all('h3', {'class':'title'})
        thread_titles_and_links = dict(map(lambda subtopic: (subtopic.text.strip(), subtopic.find('a') ['href']), subtopics))
        thread_counter = 0
        for st_key, st_value in thread_titles_and_links.items():
            thread_counter = thread_counter + 1
            if thread_counter > 3:
                break
            thread_url = Canadavisa_url + st_value
            thread_title = st_key
            thread_content = bs(requests.get(thread_url).text, 'html.parser')

            last_page = 1
            try:

                last_page = int(re.search('Page \d* of (\d*)', thread_content.select('span.pageNavHeader')[0].text).group(1))
            except Exception as e: 
                print(e) 

            for x in range(1, last_page + 1):
                if x > 3:
                    break

                thread_url = Canadavisa_url + st_value
                if x > 1:
                    thread_url = thread_url + 'page-' + str(x)
                print(thread_url)
                thread_content = bs(requests.get(thread_url).text, 'html.parser')
                messages = thread_content.select('div.messageContent')
                m_ids = thread_content.select('li.message')
                message_urls = thread_content.select('div.uix_message a.datePermalink')
                message_likes = thread_content.select('div.uix_message div.uix_userTextInner > dl:nth-of-type(2) > dd')
                message_dates = thread_content.select('span.DateTime')
                message_usernames = thread_content.select('a.username')
                message_list = list(map(lambda message: (message.find('blockquote').decode_contents().strip()), messages))
                usernames_list = list(map(lambda a: a.text.strip(), message_usernames))
                for idx, m in enumerate(message_list):
                    message_id = ""
                    try:
                        message_id = re.search('post-(\d*)', m_ids[idx]['id']).group(1)
                    except IndexError:
                        message_id = ""
                    message_ids.append(message_id)
                    link = ""
                    try:
                        link = 'https://www.canadavisa.com/' + message_urls[idx]['href']
                    except IndexError:
                        link = ""
                    links.append(link)
                    message_date = ""
                    try:
                        message_date = message_dates[idx]['title']
                    except IndexError:
                        message_date = ""
                    date_contents.append(message_date)
                    message_username = ""
                    try:
                        message_username = usernames_list[idx]
                    except IndexError:
                        message_username = ""
                    username_contents.append(message_username)

                    message_contents.append(m.replace('\n', ""))

                    countries.append("NA")
                    like = ""
                    try:
                        like = message_likes[idx].text
                    except IndexError:
                        like = ""
                    likes.append(like)
                    topic_contents.append(topic_title)
                    thread_contents.append(thread_title)
    except:
        print("Can't find this page... Moving on")
print("Finished")


inside loop Immigration to Canada .#immigration-to-canada.9
Processing  Express Entry / Expression of Interest
https://www.canadavisa.com/canada-immigration-discussion-board/threads/express-entry-step-by-step-instructions.273928/
https://www.canadavisa.com/canada-immigration-discussion-board/threads/express-entry-step-by-step-instructions.273928/page-2
https://www.canadavisa.com/canada-immigration-discussion-board/threads/express-entry-step-by-step-instructions.273928/page-3
https://www.canadavisa.com/canada-immigration-discussion-board/threads/summary-of-eligibility-requirements-for-all-express-entry-pnp-streams.432937/
https://www.canadavisa.com/canada-immigration-discussion-board/threads/summary-of-eligibility-requirements-for-all-express-entry-pnp-streams.432937/page-2
https://www.canadavisa.com/canada-immigration-discussion-board/threads/summary-of-eligibility-requirements-for-all-express-entry-pnp-streams.432937/page-3
https://www.canadavisa.com/canada-immigration-discussion-boar

In [131]:
# Create empty frame
#base_data_frame = pd.DataFrame({ "ID": message_ids, "Link": links, "Date": date_contents, "Username": username_contents, "Message": message_contents, "Country": [], "Like": [], "Topic": topic_contents, "Thread": thread_contents})
base_data_frame = pd.DataFrame({ "ID": message_ids, 
                                "Link": links, 
                                "Date": date_contents,
                                "Username": username_contents, 
                                "Message": message_contents, 
                                "Country": countries, 
                                "Like": likes, 
                                "Topic": topic_contents, 
                                "Thread": thread_contents})

print('Finished')

Finished


In [132]:
len(base_data_frame)

135

In [133]:
# from https://github.com/pandas-dev/pandas/issues/22610
import csv
with open("can-forum.csv", "w", errors='surrogatepass') as _file:
    writer = csv.writer(_file)
    #writer.writerow(row)
    writer.writerow(base_data_frame.columns.values)
    for index, row in base_data_frame.iterrows():
        writer.writerow(row)